In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


In [6]:
from pathlib import Path
from netCDF4 import Dataset

data_folder = Path('/Users/kristiinajoon/Desktop/4th_year/4th_year_project/Project/top_data/GEBCO_2019/')
file2open = data_folder / 'GEBCO_2019.nc' #file with location
nc_GEBCO = Dataset(file2open, 'r')
raw_lat = nc_GEBCO.variables['lat'][:] # import lat, in degrees N
raw_lon = nc_GEBCO.variables['lon'] [:]# import lat, in degrees E 
# import lat, in m as height above reference ellipsoid
raw_elevation = nc_GEBCO.variables['elevation'] [:]

# Define domain
#Indices for 35.5-41.4N & -22 - -14.5E obtained in MatLAB
lat_Prt = raw_lat[30120:31536] 
lon_Prt = raw_lon[37920:39720] 
bathy_Prt = raw_elevation[30120:31536, 37920:39720]
# len_lat = len(lat_Prt)
len_lon = len(lon_Prt)



In [7]:
import numpy as np
from math import pi, sin, cos, sqrt

def wgs84(): #WGSS84 coordinate system with Greenwich as lon = 0
    # set semi-major axis of the oblate spheroid Earth, in m
    a = 6378137.0
    # set semi-minor axis of the oblate spheroid Earth, in m
    b = 6356752.314245
    # calculate inverse flattening f
    f = a/(a-b)
    # calculate squared eccentricity e
    e_2 = (a**2-b**2)/a**2
    return(a,b,e_2,f)

def geograph_to_geocent(theta):
    # https://en.wikipedia.org/wiki/Latitude#Geocentric_latitude
    e_2 = wgs84()[2] # returns the 3rd item from function ouputs 
    theta = np.rad2deg(np.arctan((1 - e_2) * np.tan(np.deg2rad(theta))))
    return theta

lat_Prt_cnt = geograph_to_geocent(lat_Prt) 


In [8]:
import numpy as np
from math import pi, sin, cos, sqrt

# Reference surface will be the Earth as defined by the WGS84 ellipsoid
def radius_cnt(lat):
    a = wgs84()[0]
    b = wgs84()[1]
    # Calculate radius for reference ellipsoid
    # in m 
    lat = pi/180*lat
    # for i in range(len(lat)): 
    r_cnt = np.sqrt((a**2*(np.cos(lat)**2)) + (b**2*(np.sin(lat)**2)))
    return(r_cnt)

# Calculate radius of reference surface at geocentric latitudes
r_cnt = radius_cnt(lat_Prt_cnt) 
r_cnt = np.array([r_cnt,]*len_lon).transpose()
# Calculate the radius for each bathymetry data point, in m 
r_cnt_bathy = r_cnt + bathy_Prt 
# Calculate the colatitude to define a spherical coordinate system
colat_Prt_cnt = 90 - lat_Prt_cnt

# Variables in spherical coordinates: r_cnt_bathy, lat_Prt_cnt, lon_Prt


In [9]:
np.savetxt('r_cnt_bathy.txt', r_cnt_bathy)

In [10]:
import numpy as np
from math import pi, sin, cos, sqrt

# Reference surface will be the Earth as defined by the WGS84 ellipsoid
def radius_cnt(lat):
    a = wgs84()[0]
    b = wgs84()[1]
    # Calculate radius for reference ellipsoid
    # in m 
    lat = pi/180*lat
    # for i in range(len(lat)): 
    r_cnt = np.sqrt((a**2*(np.cos(lat)**2)) + (b**2*(np.sin(lat)**2)))
    return(r_cnt)

# Calculate radius of reference surface at geocentric latitudes
r_cnt = radius_cnt(lat_Prt_cnt) 
r_cnt = np.array([r_cnt,]*len_lon).transpose()
# Calculate the radius for each bathymetry data point, in m 
r_cnt_bathy = r_cnt + bathy_Prt 
# Calculate the colatitude to define a spherical coordinate system
colat_Prt_cnt = 90 - lat_Prt_cnt

np.savetxt('r_cnt_bathy.txt', r_cnt_bathy)
np.savetxt('colat_Prt_cnt.txt', colat_Prt_cnt)
np.savetxt('lon_Prt.txt', lon_Prt)


# Variables in spherical coordinates: r_cnt_bathy, colat_Prt_cnt, lon_Prt


In [11]:
r_cnt_bathy_ls = [line.rstrip('\n') for line in open('r_cnt_bathy.txt')]

In [12]:
import numpy as np
from math import sin, cos, pi

# Read input text files as lists
r_cnt_bathy_ls = [line.rstrip('\n') for line in open('r_cnt_bathy.txt')]
colat_Prt_cnt_ls = [line.rstrip('\n') for line in open('colat_Prt_cnt.txt')]
lon_Prt_ls = [line.rstrip('\n') for line in open('lon_Prt.txt')]

# Convert lists to input arrays
r_cnt_bathy = np.asarray(r_cnt_bathy_ls)
colat_Prt_cnt = np.asarray(colat_Prt_cnt_ls)
lon_Prt = np.asarray(lon_Prt_ls)

In [13]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon
    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)*np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)

# sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


In [14]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon
    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)*np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)

sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)



TypeError: ufunc 'multiply' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [15]:
# Convert lists to input arrays
r_cnt_bathy = np.asarray(r_cnt_bathy_ls, dtype = 'float64')
colat_Prt_cnt = np.asarray(colat_Prt_cnt_ls, dtype = 'float64')
lon_Prt = np.asarray(lon_Prt_ls, dtype = 'float64')

ValueError: could not convert string to float: '6.365973282435107045e+06 6.365972038294482045e+06 6.365970096888232045e+06 6.365967483118700795e+06 6.365964336634325795e+06 6.365960880579638295e+06 6.365957397181200795e+06 6.365953919642138295e+06 6.365950265833544545e+06 6.365946415247607045e+06 6.365942511927294545e+06 6.365938510950732045e+06 6.365934173060107045e+06 6.365929421106982045e+06 6.365924361048388295e+06 6.365918988489794545e+06 6.365913185755419545e+06 6.365907016321825795e+06 6.365900702356982045e+06 6.365894185267138295e+06 6.365887155481982045e+06 6.365879613489794545e+06 6.365872029505419545e+06 6.365864568567919545e+06 6.365857349329638295e+06 6.365851879603075795e+06 6.365851033899950795e+06 6.365854852747607045e+06 6.365860581751513295e+06 6.365866956751513295e+06 6.365874279993700795e+06 6.365882902064013295e+06 6.365893222864794545e+06 6.365907024622607045e+06 6.365926717493700795e+06 6.365950829798388295e+06 6.365973122767138295e+06 6.365985437220263295e+06 6.

In [16]:
r_cnt_bathy = np.asarray(r_cnt_bathy_ls, dtype = 'float')

ValueError: could not convert string to float: '6.365973282435107045e+06 6.365972038294482045e+06 6.365970096888232045e+06 6.365967483118700795e+06 6.365964336634325795e+06 6.365960880579638295e+06 6.365957397181200795e+06 6.365953919642138295e+06 6.365950265833544545e+06 6.365946415247607045e+06 6.365942511927294545e+06 6.365938510950732045e+06 6.365934173060107045e+06 6.365929421106982045e+06 6.365924361048388295e+06 6.365918988489794545e+06 6.365913185755419545e+06 6.365907016321825795e+06 6.365900702356982045e+06 6.365894185267138295e+06 6.365887155481982045e+06 6.365879613489794545e+06 6.365872029505419545e+06 6.365864568567919545e+06 6.365857349329638295e+06 6.365851879603075795e+06 6.365851033899950795e+06 6.365854852747607045e+06 6.365860581751513295e+06 6.365866956751513295e+06 6.365874279993700795e+06 6.365882902064013295e+06 6.365893222864794545e+06 6.365907024622607045e+06 6.365926717493700795e+06 6.365950829798388295e+06 6.365973122767138295e+06 6.365985437220263295e+06 6.

In [17]:
def save_load(opt,filename):
    global calc
    if opt == "save":
        f = file(filename, 'wb')
        pickle.dump(calc, f, 2)
        f.close
        print 'Data saved'
    elif opt == "load":
        f = file(filename, 'rb')
        calc = pickle.load(f)
    else:
        print 'Invalid option, can either "save" or "load"'

        

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Data saved')? (<ipython-input-17-414eaaf54144>, line 7)

In [18]:
def save_load(opt,filename):
    global calc
    if opt == "save":
        f = file(filename, 'wb')
        pickle.dump(calc, f, 2)
        f.close
        print('Data saved')
    elif opt == "load":
        f = file(filename, 'rb')
        calc = pickle.load(f)
    else:
        print('Invalid option, can either "save" or "load"')
        

In [19]:
def save_load(opt):
    global calc
    if opt == "save":
        f = file(filename, 'wb')
        pickle.dump(calc, f, 2)
        f.close
        print('Data saved')
    elif opt == "load":
        f = file(filename, 'rb')
        calc = pickle.load(f)
    else:
        print('Invalid option, can either "save" or "load"')

In [20]:
save_load('save')

NameError: name 'file' is not defined

In [21]:
import numpy as np

np.savetxt('r_cnt_bathy', r_cnt_bathy)
np.savetxt('colat_Prt_cnt', colat_Prt_cnt)
np.savetxt('lon_Prt', lon_Prt)

# Variables in spherical coordinates: r_cnt_bathy, colat_Prt_cnt, lon_Prt


TypeError: Mismatch between array dtype ('<U44999') and format specifier ('%.18e')

In [22]:
import numpy as np

np.save('r_cnt_bathy', r_cnt_bathy)
np.save('colat_Prt_cnt', colat_Prt_cnt)
np.save('lon_Prt', lon_Prt)

# Variables in spherical coordinates: r_cnt_bathy, colat_Prt_cnt, lon_Prt


In [23]:
import numpy as np
from math import sin, cos, pi

# Convert lists to input arrays
r_cnt_bathy = np.load('r_cnt_bathy.npy')
colat_Prt_cnt = np.load('colat_Prt_cnt.npy')
lon_Prt = np.load('lon_Prt.npy')

In [24]:
prin(lon_Prt[0,-1])

NameError: name 'prin' is not defined

In [25]:
print(lon_Prt[0,-1])

IndexError: too many indices for array

In [26]:
print(lon_Prt[0])

-2.199791666666666856e+01


In [27]:
type(lon_Prt[0])

numpy.str_

In [28]:
r_cnt_bathy = r_cnt_bathy_ls.astype(np.float)

AttributeError: 'list' object has no attribute 'astype'

In [29]:
import numpy as np
from math import sin, cos, pi

# Load saved input files
r_cnt_bathy_ls = np.load('r_cnt_bathy.npy')
colat_Prt_cnt_ls = np.load('colat_Prt_cnt.npy')
lon_Prt_ls = np.load('lon_Prt.npy')

# Convert from np.str to np.float
r_cnt_bathy = r_cnt_bathy_ls.astype(np.float)
colat_Prt_cnt = colat_Prt_cnt_ls.astype(np.float)
lon_Prt = lon_Prt_ls.astype(np.float)

ValueError: could not convert string to float: '6.365973282435107045e+06 6.365972038294482045e+06 6.365970096888232045e+06 6.365967483118700795e+06 6.365964336634325795e+06 6.365960880579638295e+06 6.365957397181200795e+06 6.365953919642138295e+06 6.365950265833544545e+06 6.365946415247607045e+06 6.365942511927294545e+06 6.365938510950732045e+06 6.365934173060107045e+06 6.365929421106982045e+06 6.365924361048388295e+06 6.365918988489794545e+06 6.365913185755419545e+06 6.365907016321825795e+06 6.365900702356982045e+06 6.365894185267138295e+06 6.365887155481982045e+06 6.365879613489794545e+06 6.365872029505419545e+06 6.365864568567919545e+06 6.365857349329638295e+06 6.365851879603075795e+06 6.365851033899950795e+06 6.365854852747607045e+06 6.365860581751513295e+06 6.365866956751513295e+06 6.365874279993700795e+06 6.365882902064013295e+06 6.365893222864794545e+06 6.365907024622607045e+06 6.365926717493700795e+06 6.365950829798388295e+06 6.365973122767138295e+06 6.365985437220263295e+06 6.

In [30]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon
    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)*np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)

sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


TypeError: ufunc 'multiply' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [31]:
import numpy as np
from math import pi, sin, cos, sqrt
from obspy.geodetics import gps2dist_azimuth
from pathlib import Path
from netCDF4 import Dataset

# Laura Ermert, modified by Kristiina Joon

# In [143]: lat_Prt_cnt[0]                                                                                                                                               
# Out[143]: 35.320337657545913
# In [144]: lat_Prt_cnt[-1]                                                                                                                                              
# Out[144]: 41.20709287604776
# In [145]: lat_Prt[0]                                                                                                                                                   
# Out[145]: 35.502083333333331
# In [146]: lat_Prt[-1]                                                                                                                                                  
# Out[146]: 41.397916666666674

#%%  Import global elevation files using Dataset
from pathlib import Path
from netCDF4 import Dataset

data_folder = Path('/Users/kristiinajoon/Desktop/4th_year/4th_year_project/Project/top_data/GEBCO_2019/')
file2open = data_folder / 'GEBCO_2019.nc' #file with location
nc_GEBCO = Dataset(file2open, 'r')
raw_lat = nc_GEBCO.variables['lat'][:] # import lat, in degrees N
raw_lon = nc_GEBCO.variables['lon'] [:]# import lat, in degrees E 
# import lat, in m as height above reference ellipsoid
raw_elevation = nc_GEBCO.variables['elevation'] [:]

# Define domain
#Indices for 35.5-41.4N & -22 - -14.5E obtained in MatLAB
lat_Prt = raw_lat[30120:31536] 
lon_Prt = raw_lon[37920:39720] 
bathy_Prt = raw_elevation[30120:31536, 37920:39720]
# len_lat = len(lat_Prt)
len_lon = len(lon_Prt)

In [32]:
import numpy as np
from math import pi, sin, cos, sqrt

def wgs84(): #WGSS84 coordinate system with Greenwich as lon = 0
    # set semi-major axis of the oblate spheroid Earth, in m
    a = 6378137.0
    # set semi-minor axis of the oblate spheroid Earth, in m
    b = 6356752.314245
    # calculate inverse flattening f
    f = a/(a-b)
    # calculate squared eccentricity e
    e_2 = (a**2-b**2)/a**2
    return(a,b,e_2,f)

def geograph_to_geocent(theta):
    # https://en.wikipedia.org/wiki/Latitude#Geocentric_latitude
    e_2 = wgs84()[2] # returns the 3rd item from function ouputs 
    theta = np.rad2deg(np.arctan((1 - e_2) * np.tan(np.deg2rad(theta))))
    return theta

lat_Prt_cnt = geograph_to_geocent(lat_Prt) 

#%% Express bathymetry as distance from the centre of the Earth
import numpy as np
from math import pi, sin, cos, sqrt

# Reference surface will be the Earth as defined by the WGS84 ellipsoid
def radius_cnt(lat):
    a = wgs84()[0]
    b = wgs84()[1]
    # Calculate radius for reference ellipsoid
    # in m 
    lat = pi/180*lat
    # for i in range(len(lat)): 
    r_cnt = np.sqrt((a**2*(np.cos(lat)**2)) + (b**2*(np.sin(lat)**2)))
    return(r_cnt)

# Calculate radius of reference surface at geocentric latitudes
r_cnt = radius_cnt(lat_Prt_cnt) 
r_cnt = np.array([r_cnt,]*len_lon).transpose()
# Calculate the radius for each bathymetry data point, in m 
r_cnt_bathy = r_cnt + bathy_Prt 
# Calculate the colatitude to define a spherical coordinate system
colat_Prt_cnt = 90 - lat_Prt_cnt

In [33]:
sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)

ValueError: operands could not be broadcast together with shapes (1416,1800) (1416,) 

In [34]:
sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt) [0]

ValueError: operands could not be broadcast together with shapes (1416,1800) (1416,) 

In [35]:
sph_to_cartesian(r_cnt_bathy[0], colat_Prt_cnt[0], lon_Prt[0]) 

(masked_array(data = [ 4816052.83747589  4816051.89624558  4816050.42751257 ...,
   4816575.30230302  4816569.44584098  4816564.71678666],
              mask = False,
        fill_value = 1e+20),
 masked_array(data = [-1945607.95217202 -1945607.5719301  -1945606.97858557 ...,
  -1945819.01956615 -1945816.65364959 -1945814.74318765],
              mask = False,
        fill_value = 1e+20),
 masked_array(data = [ 3680470.15960775  3680469.44031122  3680468.31789247 ...,
   3680869.43184802  3680864.95628795  3680861.34230283],
              mask = False,
        fill_value = 1e+20))

In [36]:
(x,y,z) = sph_to_cartesian(r_cnt_bathy[0], colat_Prt_cnt[0], lon_Prt[0])

In [37]:
(n,m) = r_cnt_bathy.shape

In [38]:
if len(colat) = n:
        colat = np.array([colat]*m)
    elif len(colat) = m: 
        colat = np.array([colat]*n).conj()transpose()
    else:
        print('Input arguments must have at least one similar array dimension')
        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [40]:
if len(colat) = n:
        colat = np.array([colat]*m)
        elif len(colat) = m: 
            colat = np.array([colat]*n).conj()transpose()
        else:
            print('Input arguments must have at least one similar array dimension')
            

SyntaxError: invalid syntax (<ipython-input-40-97a0c3479ed0>, line 1)

In [41]:
k = len(lat_Prt_cnt)

In [42]:
if len(lat_Prt) = n:
        lat_Prt = np.array([lat_Prt]*m)
        elif len(lat_Prt) = m: 
            lat_Prt = np.array([lat_Prt]*n).conj()transpose()
        else:
            print('Input arguments must have at least one similar array dimension')

SyntaxError: invalid syntax (<ipython-input-42-91cc759f0d78>, line 1)

In [43]:
if len(lat_Prt) == n:
        lat_Prt = np.array([lat_Prt]*m)
        elif len(lat_Prt) == m: 
            lat_Prt = np.array([lat_Prt]*n).conj()transpose()
        else:
            print('Input arguments must have at least one similar array dimension')

SyntaxError: invalid syntax (<ipython-input-43-855a034a4d92>, line 3)

In [44]:
if len(lat_Prt) == n:
    lat_Prt = np.array([lat_Prt]*m)
elif len(lat_Prt) == m: 
    lat_Prt = np.array([lat_Prt]*n).conj()transpose()
else:
    print('Input arguments must have at least one similar array dimension')

SyntaxError: invalid syntax (<ipython-input-44-7dc6f256ae9e>, line 4)

In [45]:
if len(lat_Prt) == n:
    lat_Prt = np.array([lat_Prt]*m)
elif len(lat_Prt) == m: 
    lat_Prt = np.array([lat_Prt]*n).transpose()
else:
    print('Input arguments must have at least one similar array dimension')

In [46]:
len(lat_Prt_cnt)

1416

In [47]:
r = np.array([1:10]*10)

SyntaxError: invalid syntax (<ipython-input-47-0f42028f7338>, line 1)

In [48]:
r = np.arange(1,11, dtype=Float)

NameError: name 'Float' is not defined

In [49]:
r = np.arange(1,11)

In [50]:
r = np.array([r]*10)

In [51]:
r = np.arange(1,11)

In [52]:
r = np.array([r]*20)

In [53]:
colat = np.arange(1,11)
lon = np.arange(1,21)

(n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj()transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    if len(lon) == n:
    lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj()transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

IndentationError: unexpected indent (<ipython-input-53-f6958623facd>, line 6)

In [54]:
colat = np.arange(1,11)
lon = np.arange(1,21)

(n,m) = r.shape

if len(colat) == n:
    colat = np.array([colat]*m)
elif len(colat) == m: 
    colat = np.array([colat]*n).conj()transpose()
else:
    print('Input arguments must have at least one similar array dimension')

if len(lon) == n:
lon = np.array([lon]*m)
elif len(lon) == m: 
    lon = np.array([lon]*n).conj()transpose()
else:
    print('Input arguments must have at least one similar array dimension')

SyntaxError: invalid syntax (<ipython-input-54-822b087dbb1e>, line 9)

In [55]:
colat = np.arange(1,11)
lon = np.arange(1,21)

(n,m) = r.shape

if len(colat) == n:
    colat = np.array([colat]*m)
elif len(colat) == m: 
    colat = np.array([colat]*n).conj().transpose()
else:
    print('Input arguments must have at least one similar array dimension')

if len(lon) == n:
lon = np.array([lon]*m)
elif len(lon) == m: 
    lon = np.array([lon]*n).conj().transpose()
else:
    print('Input arguments must have at least one similar array dimension')

IndentationError: expected an indented block (<ipython-input-55-497018239020>, line 14)

In [56]:
colat = np.arange(1,11)
lon = np.arange(1,21)

(n,m) = r.shape

if len(colat) == n:
    colat = np.array([colat]*m)
elif len(colat) == m: 
    colat = np.array([colat]*n).conj().transpose()
else:
    print('Input arguments must have at least one similar array dimension')

if len(lon) == n:
    lon = np.array([lon]*m)
elif len(lon) == m: 
    lon = np.array([lon]*n).conj().transpose()
else:
    print('Input arguments must have at least one similar array dimension')

r.shape
colat.shape
lon.shape

(10, 20)

In [57]:
colat.shape

(10, 20)

In [58]:
lon.shape

(10, 20)

In [59]:
def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj()transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    if len(lon) == n:
    lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj()transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)*np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)

SyntaxError: invalid syntax (<ipython-input-59-2993c142819e>, line 18)

In [60]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    if len(lon) == n:
    lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)*np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)
    


IndentationError: expected an indented block (<ipython-input-60-340756a5daf9>, line 23)

In [61]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)*np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)

# sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


In [62]:
sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)

ValueError: operands could not be broadcast together with shapes (1416,1800) (1800,1416) 

In [63]:
shape.colat_Prt_cnt

NameError: name 'shape' is not defined

In [64]:
colat_Prt_cnt.shape

(1416,)

In [65]:
lon_Prt.shape

(1800,)

In [66]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # # Degrees to rad
    # colat = pi/180*colat 
    # lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one similar array dimension')

    # # Transform
    # x = r*np.sin(colat)*np.cos(lon)
    # y = r*np.sin(colat)*np.sin(lon)
    # z = r*np.cos(colat)

    # return(x,y,z)
    return(colat.shape, lon.shape)


sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


((1800, 1416), (1800, 1416))

In [67]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)*np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)


sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


ValueError: operands could not be broadcast together with shapes (1416,1800) (1800,1416) 

In [68]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    # Transform
    for i in len(r[:,0]):
        for j in len(r[0,:]):
            x = r*np.sin(colat)*np.cos(lon)
            y = r*np.sin(colat)*np.sin(lon)
            z = r*np.cos(colat)

    return(x,y,z)


sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


TypeError: 'int' object is not iterable

In [69]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    # Transform
    for i in range(len(r[:,0])):
        for j in range(len(r[0,:])):
            x = r*np.sin(colat)*np.cos(lon)
            y = r*np.sin(colat)*np.sin(lon)
            z = r*np.cos(colat)

    return(x,y,z)


sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


ValueError: operands could not be broadcast together with shapes (1416,1800) (1800,1416) 

In [70]:
sin(30)

-0.9880316240928618

In [71]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    # Transform
    x = r*sin(colat)*cos(lon)
    y = r*sin(colat)* sin(lon)
    z = r* cos(colat)

    return(x,y,z)


sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


TypeError: only size-1 arrays can be converted to Python scalars

In [72]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    # Transform
    x = r*sin(colat)*cos(lon.con().transpose())
    y = r*sin(colat)* sin(lon.conj().transpose())
    z = r* cos(colat)

    return(x,y,z)


sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


TypeError: only size-1 arrays can be converted to Python scalars

In [73]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    # Transform
    x = r*np.sin(colat)*np.cos(lon.con().transpose())
    y = r*np.sin(colat)* np.sin(lon.conj().transpose())
    z = r*np.cos(colat)

    return(x,y,z)


sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


ValueError: operands could not be broadcast together with shapes (1416,1800) (1800,1416) 

In [74]:
r.shape

(20, 10)

In [75]:
r = np.arange(1,11)
r = np.array([r]*21)
colat = np.arange(1,11)
lon = np.arange(1,21)

In [76]:
sph_to_cartesian(r,colat,lon)

Input arguments must have at least one identical array dimension


ValueError: operands could not be broadcast together with shapes (21,10) (10,21) 

In [77]:
r = np.arange(1,11)
r = np.array([r]*20)
colat = np.arange(1,11)
lon = np.arange(1,21)

In [78]:
sph_to_cartesian(r,colat,lon)

ValueError: operands could not be broadcast together with shapes (20,10) (10,20) 

In [79]:
r.shape
colat.shape
lon.shape

(20,)

In [80]:
if len(colat) == n:
        colat = np.array([colat]*m)
    elif len(colat) == m: 
        colat = np.array([colat]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m)
    elif len(lon) == m: 
        lon = np.array([lon]*n).conj().transpose()
    else:
        print('Input arguments must have at least one identical array dimension')

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [82]:
if len(colat) == n:
    colat = np.array([colat]*m)
elif len(colat) == m: 
    colat = np.array([colat]*n).conj().transpose()
else:
    print('Input arguments must have at least one identical array dimension')

if len(lon) == n:
    lon = np.array([lon]*m)
elif len(lon) == m: 
    lon = np.array([lon]*n).conj().transpose()
else:
    print('Input arguments must have at least one identical array dimension')

In [83]:
r = np.arange(1,11)
r = np.array([r]*20)
colat = np.arange(1,11)
lon = np.arange(1,21)

In [84]:
sph_to_cartesian(r,colat,lon)

ValueError: operands could not be broadcast together with shapes (20,10) (10,20) 

In [85]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m).conj().transpose()
    elif len(colat) == m: 
        colat = np.array([colat]*n)
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m).conj().transpose()
    elif len(lon) == m: 
        lon = np.array([lon]*n)
    else:
        print('Input arguments must have at least one identical array dimension')

    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)* np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)


# sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


In [86]:

sph_to_cartesian(r,colat,lon)

(array([[ 0.01744975,  0.06978836,  0.15698396,  0.2789834 ,  0.43571234,
          0.62707526,  0.85295547,  1.11321523,  1.40769575,  1.7362173 ],
        [ 0.01744177,  0.06975647,  0.15691222,  0.27885592,  0.43551325,
          0.62678872,  0.85256573,  1.11270656,  1.40705252,  1.73542396],
        [ 0.01742849,  0.06970334,  0.15679269,  0.2786435 ,  0.43518149,
          0.62631126,  0.85191628,  1.11185895,  1.40598069,  1.73410199],
        [ 0.01740989,  0.06962897,  0.15662541,  0.2783462 ,  0.43471718,
          0.62564302,  0.85100733,  1.11067266,  1.40448059,  1.73225179],
        [ 0.01738599,  0.06953339,  0.15641041,  0.27796412,  0.43412044,
          0.62478421,  0.84983916,  1.10914804,  1.40255266,  1.72987394],
        [ 0.0173568 ,  0.06941663,  0.15614776,  0.27749736,  0.43339147,
          0.62373507,  0.84841211,  1.10728557,  1.40019751,  1.72696915],
        [ 0.01732232,  0.06927872,  0.15583756,  0.27694608,  0.43253049,
          0.62249594,  0.8467266

In [87]:

def sph_to_cartesian(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    Cartesian coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 
    lon = pi/180*lon

    # Reshape colat, lon to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m).conj().transpose()
    elif len(colat) == m: 
        colat = np.array([colat]*n)
    else:
        print('Input arguments must have at least one identical array dimension')

    if len(lon) == n:
        lon = np.array([lon]*m).conj().transpose()
    elif len(lon) == m: 
        lon = np.array([lon]*n)
    else:
        print('Input arguments must have at least one identical array dimension')

    # Transform
    x = r*np.sin(colat)*np.cos(lon)
    y = r*np.sin(colat)* np.sin(lon)
    z = r*np.cos(colat)

    return(x,y,z)


(x_Prt,y_Prt,z_Prt) = sph_to_cartesian(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


In [88]:
import numpy as np
from math import pi

def sph_to_cylindrical(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    cylindrical coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 

    # Reshape colat to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m).conj().transpose()
    elif len(colat) == m: 
        colat = np.array([colat]*n)
    else:
        print('Radius and colatitude must have at least one identical array dimension')

    # Transform
    s = r*np.sin(colat)
    phi = lon
    z = r*np.cos(colat)
    
    return(s,phi,z)

# sph_to_cylindrical(r_cnt_bathy, colat_Prt_cnt, lon_Prt)




In [89]:
import numpy as np
from math import pi

def sph_to_cylindrical(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    cylindrical coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 

    # Reshape colat to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m).conj().transpose()
    elif len(colat) == m: 
        colat = np.array([colat]*n)
    else:
        print('Radius and colatitude must have at least one identical array dimension')

    # Transform
    s = r*np.sin(colat)
    phi = lon
    z = r*np.cos(colat)
    
    return(s,phi,z)

sph_to_cylindrical(r_cnt_bathy, colat_Prt_cnt, lon_Prt)




(masked_array(data =
  [[ 5194204.00416797  5194202.98903329  5194201.40497699 ...,
    5194767.49235813  5194761.17605313  5194756.07567868]
  [ 5193922.80438847  5193921.64748128  5193919.76910063 ...,
    5194500.70781458  5194493.92174121  5194489.97335721]
  [ 5193642.98662115  5193641.73775148  5193639.55591442 ...,
    5194239.73558889  5194232.52321716  5194228.21770412]
  ..., 
  [ 4789083.05540569  4789090.35778876  4789095.8424751  ...,
    4788096.35340825  4788098.65588535  4788095.88718144]
  [ 4788774.40517608  4788783.81012509  4788791.31814028 ...,
    4787792.80759311  4787790.91929258  4787790.36235413]
  [ 4788466.84788874  4788480.02180427  4788489.79699506 ...,
    4787487.49040499  4787482.23913282  4787484.40613209]],
              mask =
  False,
        fill_value = 1e+20),
 masked_array(data = [-21.99791667 -21.99375    -21.98958333 ..., -14.51041667 -14.50625
  -14.50208333],
              mask = False,
        fill_value = 1e+20),
 masked_array(data =
  [[ 

In [90]:
import numpy as np
from math import pi

def sph_to_cylindrical(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    cylindrical coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 

    # Reshape colat to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m).conj().transpose()
    elif len(colat) == m: 
        colat = np.array([colat]*n)
    else:
        print('Radius and colatitude must have at least one identical array dimension')

    # Transform
    s = r*np.sin(colat)
    phi = lon
    z = r*np.cos(colat)
    
    return(s,phi,z)

(s,phi,z) = sph_to_cylindrical(r_cnt_bathy, colat_Prt_cnt, lon_Prt)




In [91]:
del s
del phi
del z 

In [92]:
import numpy as np
from math import pi

def sph_to_cylindrical(r,colat,lon):
    """
    Function to transform from spherical polar coordinates to 
    cylindrical coordinates. Function takes three arguments: 
    radius from origin (0,0,0), colatitude (inclination), 
    longitude (azimuth). Angles in degrees.
    """ 
    # Degrees to rad
    colat = pi/180*colat 

    # Reshape colat to match shape of r
    (n,m) = r.shape

    if len(colat) == n:
        colat = np.array([colat]*m).conj().transpose()
    elif len(colat) == m: 
        colat = np.array([colat]*n)
    else:
        print('Radius and colatitude must have at least one identical array dimension')

    # Transform
    s = r*np.sin(colat)
    phi = lon
    z = r*np.cos(colat)
    
    return(s,phi,z)

(s_Prt,phi_Prt,z_Prt) = sph_to_cylindrical(r_cnt_bathy, colat_Prt_cnt, lon_Prt)


